# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Постройте модель с максимально большим значением *accuracy*.

## Откройте и изучите файл

Импортируем нужные нам библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(data.info())#все данные заполнены, пропусков нет
display(data.describe())
display(data.duplicated().sum())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


0

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data['calls'] =data['calls'].astype('int64')
data['messages'] = data['messages'].astype('int64')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null float64
messages    3214 non-null int64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


**Вывод** 

Познакомившись с данными видим, что пропусков нет, данные адекватные.

## Разбивка данных на выборки

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.

In [6]:
features = data.drop(['is_ultra'], axis = 1)# выделим признаки модели
target = data['is_ultra']#выделим целевой признак

In [7]:
features_train, features_valid_0, target_train, target_valid_0  = train_test_split(features, target, test_size=0.40, random_state=12345) #  разделите данные на обучающую и валидационную выборки(которую разобьем на валидационную и тестовую выборки)
features_valid, features_test, target_valid, target_test  = train_test_split(features_valid_0, target_valid_0, test_size=0.50, random_state=12345)
features_train, target_train# тренировочные данные
features_valid, target_valid# валидационная выборка
features_test, target_test# тестовая выборка

(      calls  minutes  messages   mb_used
 160      61   495.11         8  10891.23
 2498     80   555.04        28  28083.58
 1748     87   697.23         0   8335.70
 1816     41   275.80         9  10032.39
 1077     60   428.49        20  29389.52
 ...     ...      ...       ...       ...
 2401     55   446.06        79  26526.28
 2928    102   742.65        58  16089.24
 1985     52   349.94        42  12150.72
 357      39   221.18        59  17865.23
 2313     40   301.03       102   6057.63
 
 [643 rows x 4 columns],
 160     0
 2498    0
 1748    0
 1816    0
 1077    1
        ..
 2401    0
 2928    1
 1985    0
 357     0
 2313    0
 Name: is_ultra, Length: 643, dtype: int64)

In [8]:
sd = [features_train, target_train,features_valid, target_valid,features_test, target_test]

for s in sd:
    display(s.shape)

(1928, 4)

(1928,)

(643, 4)

(643,)

(643, 4)

(643,)

**Вывод**

Данные разбиты на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. При разбитие данных на выбоорки, данные не потеряли.

## Исследование модели

Так как перед нами качественный признак,а не количественный мы можем построить следующие модели: 
 
* дерево решений 
  
* случайный лес 
  
* логистическая регрессия 
 


***СЛУЧАЙНЫЙ ЛЕС***

In [9]:
best_indicator = None
best_model = None
best_result = 0
best_est = 0
best_depth = 0

indicator= ['gini','entropy']

for indic in indicator: 
    for est in range(10, 51, 10):
        for depth in range (1, 11):
            model = RandomForestClassifier(random_state=12345, n_estimators= est,max_depth=depth,criterion = indic) # обучим модель с заданным количеством деревьев
            model.fit(features_train,target_train) # обучим модель на тренировочной выборке
            result = model.score(features_valid,target_valid) # посчитаем качество модели на валидационной выборке
            if result > best_result:
                best_model = model# сохранм наилучшую модель
                best_result = result#  сохраним наилучшее значение метрики accuracy на валидационных данных
                best_est = est # сохраним лучшее количество дереьвьев в модели
                best_depth = depth
                best_indicator = indic

                
display("Accuracy наилучшей модели на валидационной выборке:", best_result)
display("Наилучшее количество деревьев в модели на валидационной выборке:", best_est)
display("Наилучшая максимальной глубиной на валидационной выборке:", best_depth)
display("Наилучший индикатор:", best_indicator)


display(best_model)

'Accuracy наилучшей модели на валидационной выборке:'

0.8087091757387247

'Наилучшее количество деревьев в модели на валидационной выборке:'

40

'Наилучшая максимальной глубиной на валидационной выборке:'

8

'Наилучший индикатор:'

'gini'

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

***ДЕРЕВО РЕШЕНИЙ***

In [10]:
best_model_2 = None
best_result_2 = 0
best_depth_2 = 0

for depth in range(1, 6):
    model_2 = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model_2.fit(features_train, target_train)
    predictions_valid = model_2.predict(features_valid)
    result_2 = accuracy_score(target_valid, predictions_valid)
    if result_2 > best_result_2:
        best_model_2 = model_2# сохранм наилучшую модель
        best_result_2 = result_2#  сохраним наилучшее значение метрики accuracy на валидационных данных
        best_depth_2 = depth# сохраним лучшее количество дереьвьев в модели

display("Accuracy наилучшей модели на валидационной выборке:", best_result_2)
display("Наилучшее количество деревьев в модели на валидационной выборке:", best_depth_2)
display(best_model_2)

'Accuracy наилучшей модели на валидационной выборке:'

0.7853810264385692

'Наилучшее количество деревьев в модели на валидационной выборке:'

3

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

***ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ***

In [11]:
model_3 = LogisticRegression(random_state=12345)
model_3.fit(features_train,target_train)
result_3 = model_3.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result_3)
display(model_3)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
features_train, target_train# тренировочные данные
features_valid, target_valid# валидационная выборка
features_train_valid = features_train.append(features_valid)
target_train_valid = target_train.append(target_valid)


**Вывод**

Наилучшее качество на валидационной выборке показывает модель случйного леса, далее дерево решений и самое плохое качество показала логистическая регрессия. В модели случаного леса лучшее качество с гипперпараметрами:

* 'Наилучшее количество деревьев в модели на валидационной выборке:' 40

*  'Наилучшая максимальной глубиной на валидационной выборке:' 8

*  'Наилучший индикатор:''gini'

## Проверка модели на тестовой выборке

In [13]:
#проверим на тестовой выборке модель случайного леса
predictions_test_1 = best_model.predict(features_test)
result_test_1 = accuracy_score(target_test, predictions_test_1)
result_test_1

0.7962674961119751

In [14]:
#проверим на тестовой выборке модель дерева решений
predictions_test_2 = best_model_2.predict(features_test)
result_test_2 = accuracy_score(target_test, predictions_test_2)
result_test_2

0.7791601866251944

In [15]:
#проверим на тестовой выборке модель логистической регрессии
predictions_test_3 = model_3.predict(features_test)
result_test_3 = accuracy_score(target_test, predictions_test_3)
result_test_3

0.7402799377916018

обучим модели на тех гипперпараметрах только по валидационной и тестовой базе совместно

In [16]:
model_4 = RandomForestClassifier(random_state=12345, n_estimators= 40,max_depth=8) # обучим модель с заданным количеством деревьев
model_4.fit(features_train_valid,target_train_valid) # обучим модель на тренировочной выборке
predictions_test_4 = model_4.predict(features_test)
result_test_4 = accuracy_score(target_test, predictions_test_2) # посчитаем качество модели на валидационной выборке
result_test_4

0.7791601866251944

**Вывод** 

Обучение на тестовой и валидационной базе совместно не влияет на итоговый показатель точности

Проверим какой вес занимает каждый параметр в моделях случайного леса и дерева решений

In [17]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
fi = rf_feat_importance(best_model, features_test)
display(fi)
fi_2 = rf_feat_importance(best_model_2, features_test)
display(fi_2)


,cols,imp
3,mb_used,0.375424
2,messages,0.223809
0,calls,0.209406
1,minutes,0.191361


,cols,imp
3,mb_used,0.513756
1,minutes,0.274619
2,messages,0.193568
0,calls,0.018057


Видим,что в модели случайного леса параметр использованных мб занимает меньшую долю,чем в модели дерева решений

## Проверка модели на адекватность

In [18]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
DummyClassifier(strategy='most_frequent')
dummy_pred = dummy_clf.predict(features_test)


accuracy_score(target_test,dummy_pred)

0.6842923794712286

Если сравнивать полученный результат с результатами полученными моделью случайного леса, то получается модель лучшие предсказания, можно сделать вывод, что модель акдекватная

***Вывод*** 

В данном проекте были построены модели для задач классификации, которые выберает подходящий тариф. 

* Дерево решений, случайный лес, логистическая регрессия. 

* Данные были поделены на выборки, обучающую, валидационную и тестовую выборку.  

* Лучшую точность и на тестовой и на валидационной выборке показала модель случайного леса(accuracy 0.7962). У логистической регрессии(0.74), у дерева решений(0.78).
* Проверил модель на адекватность, сравнив с готовой дамми моделью(accuracy 0.6842), и она показала более высокую точность.